# Bollinger Bands

In [2]:
import pandas as pd
from datetime import datetime

import plotly.graph_objs as go


In [3]:
# Data Prep.
np_df = pd.read_csv("./data/bollinger_bands_data.csv", index_col=0)
np_df['date'] = pd.to_datetime(np_df['date']).dt.date
np_df.head(5)

,date,billing_counts,nonpayment_counts,np_pct
0,2023-09-01,1146,328,0.286213
1,2023-09-04,1099,332,0.302093
2,2023-09-08,1030,324,0.314563
3,2023-09-12,1126,352,0.312611
4,2023-09-16,1564,528,0.337596


In [4]:
def bollinger_data_calc(df, date_col, metric_col, lookback_window = 15, zscore = 2):
    # Bollinger Bands Data Calculation Function
    data_df = df.copy()
    
    # Sort the dataframe by the date_col column
    data_df = data_df.sort_values(by=[date_col], ascending=True,  ignore_index=True)

    # Calculate the n-period Simple Moving Average (SMA)
    data_df['SMA'] = data_df[metric_col].rolling(window=lookback_window).mean()

    # Calculate the n-period Standard Deviation (SD)
    data_df['SD'] = data_df[metric_col].rolling(window=lookback_window).std()
    
    for i in range(lookback_window - 1, 0, -1):
        # Calculate the n-period Simple Moving Average (SMA)
        data_df.loc[i, 'SMA'] = data_df.loc[data_df.index[0:i], [metric_col]].mean()[0]
        
        # Calculate the n-period Standard Deviation (SD)
        data_df.loc[i, 'SD'] = data_df.loc[data_df.index[0:i], [metric_col]].std()[0]

    # Calculate the Upper Bollinger Band (UB) and Lower Bollinger Band (LB)
    data_df['UB'] = data_df['SMA'] + zscore * data_df['SD']
    data_df['LB'] = data_df['SMA'] - zscore * data_df['SD']
    return data_df

In [5]:
bb_df = bollinger_data_calc(np_df, 'date', 'np_pct')
bb_df.head(5)

,date,billing_counts,nonpayment_counts,np_pct,SMA,SD,UB,LB
0,2023-09-01,1146,328,0.286213,NaN,NaN,NaN,NaN
1,2023-09-04,1099,332,0.302093,0.286213,NaN,NaN,NaN
2,2023-09-08,1030,324,0.314563,0.294153,0.011229,0.316610,0.271695
3,2023-09-12,1126,352,0.312611,0.300956,0.014209,0.329375,0.272538
4,2023-09-16,1564,528,0.337596,0.303870,0.012983,0.329836,0.277904


In [6]:
def plot_bollinger_band(df, date, metric, date_name = 'Date', metric_name = 'Metric', title = ''):
    # Bollinger Band Plotting Function.
    fig = go.Figure()

    # Add the main metric chart
    fig.add_trace(go.Scatter(x=bb_df[date], y=bb_df[metric], mode='lines', name=metric_name))

    # Add the Upper and Lower Bollinger Bands (UB) and shade the area
    fig.add_trace(go.Scatter(x=bb_df[date], y=bb_df['UB'], mode='lines', name='Upper Bollinger Band', line=dict(color='red')))
    fig.add_trace(go.Scatter(x=bb_df[date], y=bb_df['LB'], fill='tonexty', mode='lines', name='Lower Bollinger Band', line=dict(color='green')))

    # Add Moving Average of Metic (MA)
    fig.add_trace(go.Scatter(x=bb_df[date], y=bb_df['SMA'], mode='lines', name='Middle Bollinger Band', line=dict(color='blue')))

    # Customize the chart layout
    fig.update_layout(title=title,
                    xaxis_title=date_name,
                    yaxis_title=metric_name,
                    showlegend=True)

    # Show the chart
    fig.show()

In [7]:
plot_bollinger_band(bb_df, 
                    'date', 
                    'np_pct', 
                    metric_name = 'Non-Payment Pct', 
                    title = 'Non-Payment Pct with Bollinger Bands')